In [12]:
import cv2
import os
import numpy as np
from keras.utils import to_categorical
import zipfile

# Specify the path to your ZIP file
zip_file_path = 'images.zip'
# Specify the directory where you want to extract the contents
extracted_path = 'extracted_images'

# Create a ZipFile object
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents into the specified directory
    zip_ref.extractall(extracted_path)

# List the contents of the extracted directory
categories = os.listdir(extracted_path)
labels = [i for i in range(len(categories))]

# Now you can work with the extracted contents as needed
label_dict = dict(zip(categories, labels))  # null dict

print(label_dict)
print(categories)
print(labels)

img_size = 100
data = []   # images saved here
target = []  # labels are saved here (0 for mask and 1 for no mask)

for category in categories:
    folder_path = os.path.join(extracted_path, category)
    img_names = os.listdir(folder_path)

    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)

        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # converting to gray color
            # Converting the image into gray scale
            resized = cv2.resize(gray, (img_size, img_size))  # resizing all images to 100 by 100
            data.append(resized)  # appending resized images to the data list
            target.append(label_dict[category])  # appending labels for images (0 or 1)

        except Exception as e:
            print('Exception:', e)  # missing images can raise exceptions

if target:
    data = np.array(data) / 255.0  # normalizing images in a range 0 to 1
    data = np.reshape(data, (data.shape[0], img_size, img_size, 1))  # reshaping to a 4-dimensional array (1 since images are grayscale)
    target = np.array(target)  # converting target to a numpy array

    new_target = to_categorical(target)
    np.save('data', data)
    np.save('target', new_target)
else:
    print("No data found. Check if the images.zip file contains valid image data.")


{'images': 0}
['images']
[0]
Exception: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Exception: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

No data found. Check if the images.zip file contains valid image data.


In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

# Load data and target arrays
data = np.load('data.npy')
target = np.load('target.npy')

# Check the number of classes dynamically
num_classes = len(np.unique(target))

model = Sequential()
# Layer 1
model.add(Conv2D(200, (3, 3), input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Layer 2
model.add(Conv2D(100, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Split data for training and testing
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.1)

# Checkpoint to save the best model based on validation loss
checkpoint = ModelCheckpoint('model-{epoch:03d}.model', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

# Train the model
history = model.fit(train_data, train_target, epochs=20, callbacks=[checkpoint], validation_split=0.2)

# Evaluate the model on the test data
print(model.evaluate(test_data, test_target))


FileNotFoundError: ignored